<a href="https://colab.research.google.com/github/whoreallylovesme/JapaneseCandles/blob/main/candles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing some PySpark libraries
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

# importing some python3 libraries
import os
import shutil

In [ ]:
# без этой строчки про парсер ничего работать не будет
# (как я понял это проблемы современного спарка)
spark = SparkSession \
        .builder \
        .appName("SparkTask") \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .getOrCreate()

input_csv = "fin_sample.csv"
path = input("Enter PATH to CSV file or just press ENTER for std df: ")
if (path):
    input_csv = path

In [ ]:
# reading df
schema = StructType([
    StructField("#SYMBOL", StringType(), True),
    StructField("SYSTEM", StringType(), True),
    StructField("MOMENT", StringType(), True),
    StructField("ID_DEAL", LongType(), True),
    StructField("PRICE_DEAL", DoubleType(), True),
    StructField("VOLUME", LongType(), True),
    StructField("OPEN_POS", LongType(), True),
    StructField("DIRECTION", StringType(), True),
])
df = spark.read.csv(input_csv, header=True, schema=schema)
df.show(5)

In [ ]:
# just setting std options
options = {}
options["candle.width"] = 300000
options["candle.date.from"] = 19000101
options["candle.date.to"] = 20200101
options["candle.time.from"] = 1000
options["candle.time.to"] = 1800

# parsing xml config file from command line argument
xml_file_path = input(
    "Enter PATH to XML config file or just press ENTER for std options: "
)
if (xml_file_path):
    tree = xml.etree.ElementTree.parse(xml_file_path)
    root = tree.getroot()
    for prop in root.findall('property'):
        name = prop.find('name').text
        value = prop.find('value').text
        options[name] = value

In [ ]:
# convert to TIMESTAMP format
df = df.withColumn("MOMENT", F.to_timestamp("MOMENT", "yyyyMMddHHmmssSSS"))

# filtering according options
df = df.filter(
    (F.hour("MOMENT") * 100 + F.minute("MOMENT") >= options["candle.time.from"]) &
    (F.hour("MOMENT") * 100 + F.minute("MOMENT") <= options["candle.time.to"])
)
df.show(5)

In [ ]:
# making candlesticks

# splitting by time windows
window_spec = F.window("MOMENT", f"{options['candle.width']} milliseconds")

# group by #SYMBOL and time windows
# then aggregate it for answer according aliases
candlestick_df = df.groupBy("#SYMBOL", window_spec).agg(
    F.first("PRICE_DEAL").alias("OPEN"),
    F.max("PRICE_DEAL").alias("HIGH"),
    F.min("PRICE_DEAL").alias("LOW"),
    F.last("PRICE_DEAL").alias("CLOSE"),
    F.sum("VOLUME").alias("Volume")
)

# select required columns for answer
candlestick_df = candlestick_df.select(
    F.col("#SYMBOL"),
    F.col("window.start").alias("MOMENT"),
    "OPEN",
    "HIGH",
    "LOW",
    "CLOSE"
)

# order by time (ascending)
candlestick_df = candlestick_df.orderBy("#SYMBOL", "MOMENT")

# converting back to yyyyMMddHHmmssSSS from TIMESTAMP
candlestick_df = candlestick_df.withColumn(
    "MOMENT",
    F.date_format("MOMENT", "yyyyMMddHHmmssSSS")
)

candlestick_df.show(5)

In [ ]:
# writing answer

# selecting unique #SYMBOL
unique_symbols = candlestick_df \
                 .select("#SYMBOL") \
                 .distinct() \
                 .rdd \
                 .flatMap(lambda x: x) \
                 .collect()

output_dir = "AnswerSparkTask"

# rm -r output_dir
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

# writing answer to files
for symbol in unique_symbols:
    symbol_df = candlestick_df.filter(F.col("#SYMBOL") == symbol)
    output_path = os.path.join(output_dir, f"{symbol}.csv")
    symbol_df.write.csv(output_path, header=True, mode="overwrite")

    # print some logs
    print(f"Saved candlestick data for {symbol} to {output_path}")